In [1]:
from source.chroma_manager import ChromaDBManager
from source.llm_manager import LMStudioClient
from prompt.prompts import QuestionsToDoc
from tqdm import tqdm
import ast
import json
import logging

DB_CONFIG = LMStudioClient.load_config("config/embedding/questions_gen.yaml")
MODEL_CONFIG_PATH = "config/model_question_gen.yaml"

logging.basicConfig(filename="log/question_gen.log", level=logging.ERROR, format="%(asctime)s - %(levelname)s - %(message)s")

# Инициализация менеджеров
source_chroma = ChromaDBManager(
    storage_path=DB_CONFIG["chroma_path"],
    collection_name=DB_CONFIG["collection_name"]
)

llm = LMStudioClient(MODEL_CONFIG_PATH)  # URL настроить под свой сервер

# Извлечение всех документов
all_documents = source_chroma.collection.get()
doc_ids = all_documents.get("ids", [])
doc_texts = all_documents.get("documents", [])
doc_metadata = all_documents.get("metadatas", [])

C:\Users\Igorexy\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


e:\ImportantFiles\Documents\University\Magic App\source\llm_manager.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(


In [2]:
len(doc_ids)

549

In [9]:
all_documents['metadatas'][-1].keys()

dict_keys(['filename', 'keyword', 'questions', 'topic'])

In [ ]:
# doc_ids_batch = doc_ids[300:]
# doc_texts_batch = doc_texts[300:]
# doc_metadata_batch = doc_metadata[300:]

In [4]:
def safe_str_to_dict(s: str):
    """
    Преобразует строку в словарь, если возможно.
    В противном случае возвращает False.
    """
    try:
        result = ast.literal_eval(s)
        if isinstance(result, dict):
            return result
        return False
    except (ValueError, SyntaxError):
        return False

In [23]:
# Список ids где в метадате отсутвуют вопросы
ids_to_generate = []
for ids, metadatas in zip(doc_ids, doc_metadata):
    if 'questions' in list(metadatas.keys()):
        ids_to_generate.append(ids)
len(ids_to_generate)

300

In [ ]:
for doc_id, doc_text, metadata in tqdm(zip(doc_ids, doc_texts, doc_metadata)):
    if doc_id in ids_to_generate:
        # Генерация вопросов через LLM
        prompt = QuestionsToDoc.QUESTION_FORMATION.format(document=doc_text)
        # print(repr(prompt))  # Выведет строку в raw-формате
        questions = llm.post_completion(user_input=prompt)
        llm.clear_context() # очищаем контекст
        questions_dict = safe_str_to_dict(questions)
        if questions_dict == False:
            prompt = QuestionsToDoc.CORRECTING_DICKTIONARY.format(input=questions)
            questions = llm.post_completion(user_input=prompt)
            llm.clear_context() # очищаем контекст
            questions_dict = safe_str_to_dict(questions)
            if questions_dict == False:
                print("Словарь не сгенерирован")
                print(questions)
                logging.error(f" doc_id={doc_id}: questions_dict={questions}")
                continue
            
        # Обновление метаданных
        print("Успешно")
        print(questions_dict)
        metadata["questions"] = json.dumps(questions_dict)
        source_chroma.update_document(document_id=doc_id, new_text=doc_text, new_metadata=metadata)

0it [00:00, ?it/s]

Успешно
{'question_1': 'What are the key contributions of the proposed connectionist bi-directional RNN models in relation classification?', 'answer_1': 'The key contributions include using a combination of left and middle contexts for CNNs, introducing ranking loss for optimization, and proposing connectionist bi-directional RNNs that combine forward and backward passes to improve performance.', 'question_2': 'How does the extended middle context in CNNs enhance relation classification compared to traditional approaches?', 'answer_2': 'The extended middle context enhances relation classification by considering all parts of the sentence, paying special attention to the middle part which contains relevant information for relations. This reduces the risk of max pooling selecting distant irrelevant values and improves accuracy.'}


2it [00:31, 15.97s/it]

Успешно
{'question_1': 'What is the significance of using a step edge activation function in this approach to understanding neural networks?', 'answer_1': 'Using a step edge activation function simplifies the structure of fully connected feedforward neural networks, allowing for strong inferences about their general structure even when training against an approximation of this function.', 'question_2': 'How does the geometric decomposition of a neural network contribute to future work on deep learning algorithms?', 'answer_2': 'The geometric decomposition provides a framework that can be applied to build and manipulate networks, potentially improving training algorithms and computing homology of the network.'}


3it [00:53, 18.97s/it]

...

Успешно
{'question_1': 'What challenges does the proposed system face when analyzing conversations between multiple speakers simultaneously?', 'answer_1': 'The system currently can handle a conversation between two speakers where only one speaks at a time, but it struggles to understand if two people talk simultaneously.', 'question_2': 'How does the system perform sentiment analysis on individual speakers in a conversation?', 'answer_2': 'The system first uses speech recognition tools to transcribe audio recordings and then applies speaker discrimination methods based on MFCCs. Sentiment analysis is performed on the speaker-specific speech data to detect emotions of individual speakers.'}


248it [2:53:51, 40.53s/it]

Успешно
{'question_1': 'What challenges did the authors face when preparing their classification system for sentiment analysis, and how did they address these challenges?', 'answer_1': 'The authors faced challenges with informal language, multi-linguality, and class imbalance in the dataset. They addressed these by merging and shuffling the data, splitting it into training and development sets, and using techniques like class weighting to handle imbalanced classes.', 'question_2': "How did the use of regularizers contribute to the model's performance according to the article?", 'answer_2': 'Regularizers were used in the model to prevent overfitting by initializing weights with a glorot uniform initializer and applying W and b regularizers. This helped in better validation and test results, leading to improved generalization.'}


249it [2:54:33, 42.06s/it]

Успешно
{'question_1': 'What are the key challenges in developing robust sentiment analysis models for Arabic, and how do these differ from general sentiment analysis?', 'answer_1': 'Key challenges include high morphological complexity of the Arabic language, data scarcity, and cultural contextualization. These challenges make it difficult to train robust models compared to general sentiment analysis, which often has more comprehensive datasets and less linguistic complexity.', 'question_2': 'How does the survey suggest addressing dialectal variations in Arabic sentiment analysis research?', 'answer_2': 'The survey suggests addressing dialectal variations through dataset creation, domain adaptation, data augmentation, increased use of deep learning models like ARBERT and MARBERT, transfer learning, and multilingual models. These approaches help capture the nuances of different Arabic dialects more effectively.'}
